---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [1]:
import pandas as pd
import re

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.DataFrame(doc, columns=['content'])
df.head()

,content
0,03/25/93 Total time of visit (in minutes):\n
1,6/18/85 Primary Care Doctor:\n
2,sshe plans to move as of 7/8/71 In-Home Servic...
3,7 on 9/27/75 Audit C Score Current:\n
4,2/6/96 sleep studyPain Treatment Pain Level (N...


df['content'] = df['content'].findall('Sep')replace(['Jun', 'Jul', 'Aug', 'Sep'], ['06', '07', '08', '09'])  #can't work
df.loc[150:200]

In [ ]:
def fix_date(x):
    year = x.split('-')[0]
    year =int(year)
    if year > 2020:
        year = year - 100
        return str(year)+x[4:]
    else:
        return x

In [ ]:
import re
import pandas as pd

r1 = '(\d*\/\d*\/\d*)'
r2 = '(\d*\/\d*)'
r3 = '(\d{4})'
r4 = '(\d*\-\d*\-\d*)'
r5 = '(\d+\s\d{4})'
r6 = '(\d+\s\d+\s\d{4})'

date_form = [r1, r2, r3, r4, r5, r6]

for i in range(0, len(df)):
    for regex in date_form:
        x = re.findall(regex , df.loc[i, 'content'])
        try:
             df.loc[i,'transformed_date']= pd.Timestamp(x[0])
        except:
            continue

#df.loc[1:50]

In [ ]:
df['transformed_date'] = df['transformed_date'].astype(str)  
df['transformed_date'].iloc[99]='1983-11-14'
df['transformed_date'].iloc[271]='2008-08-01'

In [ ]:
df['transformed_date'] = df['transformed_date'].apply(lambda x : fix_date(x))
#df.dtypes
#df.info()
df.loc[0:30]

In [ ]:
for i in range(0,len(df)):
    df['content'][i]=df['content'][i].replace('January','01')
    df['content'][i]=df['content'][i].replace('February','02')
    df['content'][i]=df['content'][i].replace('March','03')
    df['content'][i]=df['content'][i].replace('April','04')
    df['content'][i]=df['content'][i].replace('May','05')
    df['content'][i]=df['content'][i].replace('June','06')
    df['content'][i]=df['content'][i].replace('July','07')
    df['content'][i]=df['content'][i].replace('August','08')
    df['content'][i]=df['content'][i].replace('September','09')
    df['content'][i]=df['content'][i].replace('October','10')
    df['content'][i]=df['content'][i].replace('November','11')
    df['content'][i]=df['content'][i].replace('December','12')
    df['content'][i]=df['content'][i].replace('Jan','01')
    df['content'][i]=df['content'][i].replace('Feb','02')
    df['content'][i]=df['content'][i].replace('Mar','03')
    df['content'][i]=df['content'][i].replace('Apr','04')
    df['content'][i]=df['content'][i].replace('May','05')
    df['content'][i]=df['content'][i].replace('Jun','06')
    df['content'][i]=df['content'][i].replace('Jul','07')
    df['content'][i]=df['content'][i].replace('Aug','08')
    df['content'][i]=df['content'][i].replace('Sep','09')
    df['content'][i]=df['content'][i].replace('Oct','10')
    df['content'][i]=df['content'][i].replace('Nov','11')
    df['content'][i]=df['content'][i].replace('Dec','12')  

In [ ]:
#01 2001
for i in range(0, len(df)):
    x = re.findall('\d+\s\d{4}' , df.loc[i, 'content'])
    x = str(x)
    print(x)
    if len(x) >2:
        y = x[5:9]+'-'+x[2:4]+'-'+'01'
        df.loc[i,'transformed_date'] = y
        print(i, y, x[5:9], x[2:4])
#df.loc[1:50]

In [ ]:
#24 01 2001
for i in range(0, len(df)):
    x = re.findall('\d+\s\d+\s\d{4}' , df.loc[i, 'content'])
    x = str(x)  
    print(x)
    if len(x) >2:
        if int(x[5:7])<= 12:
            #print(x, type(x))
            y = x[8:12]+'-'+x[5:7]+'-'+x[2:4]
            df.loc[i,'transformed_date'] = y
            print(i, y, x[8:12], x[5:7], x[2:4])
        else:
            y = x[8:12]+'-'+x[2:4]+'-'+x[5:7]
            df.loc[i,'transformed_date'] = y
            print(i, y)
#df.loc[1:50]     

#1/25/2011
for i in range(0, len(df)):
    x = re.findall('\d?\d\/\d?\d\/\d{4}' , df['content'].loc[i])
    print(x)
    if len(x)>0:
        x = str(x).split('/')
        if len(x[0])<= 3:
            a = '0'+x[0][2:4]
        else: a = x[0][2:4]
        if len(x[1])<1:
            b = '0'+x[1]
        else: b= x[1]
        y = x[2][0:4]+'-'+a+'-'+b
        df.loc[i,'transformed_date'] = y
        print(y)
  
#df.loc[1:50]     

In [ ]:
11/1984
for i in range(0, len(df)):
    x = re.findall('(\d?\d\/\d{4})' , df.loc[i, 'content'])
    #print(i, x, len(x))

    if len(x)>0:
        x = str(x).split('/')
        #print( x[0], x[1])
        if len(x[0])<=3: a = '0'+x[0][-1]
        else: a = x[0][2:4]
            
        y = x[1][:-2]+'-'+a+'-'+'01'
        df.loc[i,'transformed_date'] = y
        print(y)

In [ ]:
5/24/1990
for i in range(0, len(df)):
    x = re.findall('(\d?\d\/\d?\d\/\d{4})' , df.loc[i, 'content'])
    #print(i, x, len(x))
    if len(x)>=1:
        x = str(x).split('/')
        if len(x[0])<=3: a = '0'+x[0][-1]
        else: a = x[0][2:4]
        if len(x[1])<=1: b = '0'+x[1]
        else: b = x[1]
        y = x[2][0:4]+'-'+a+'-'+b
        df.loc[i,'transformed_date'] = y
        print(y)

In [ ]:
#06 13, 2011 
for i in range(0, len(df)):
    x = re.findall('\d+\s\d+\,\s\d{4}' , df.loc[i, 'content'])
    x = str(x)  
    #print(x)
    if len(x) >2:
        if int(x[5:7])<= 12:
            #print(x, type(x))
            y = x[9:13]+'-'+x[5:7]+'-'+x[2:4]
            df.loc[i,'transformed_date'] = y
            print(i, y, x[8:12], x[5:7], x[2:4])
        else:
            y = x[9:13]+'-'+x[2:4]+'-'+x[5:7]
            df.loc[i,'transformed_date'] = y
            print(i, y)
#df.loc[1:50]     

In [ ]:
df['transformed_date'].iloc[490]='2007-01-01'

In [ ]:
df.loc[450:500]

In [ ]:
df['transformed_date'] = pd.to_datetime(df['transformed_date'], format='%Y-%m-%d')
df = df.sort_values(by='transformed_date', ascending=True)
df = df.reset_index()
x = df['index']

In [ ]:
df.info()

In [ ]:
if df['content'].isna().any()==False:
    print('yes')

In [ ]:
if df['content'][1] == df['content'][1]:
    print('yes')

In [ ]:
df['content'] = df['content'].replace('Sep','09')

df['content'] = df['content'].replace(to_replace =['Jan','Feb','Mar','Apr','May','Jun',
                             'Jul','Aug','Sep','Oct','Nov','Dec'],
                            value = ['01','02','03','04','05','06',
                              '07', '08','09','10','11','12'], inplace=True)
#for i in range(0 , len(df)):
    #df['content'].loc[i] = df['content'].loc[i].replace('Sep','09')
   # df['content'].loc[i] = df['content'].loc[i].replace(['Jan','Feb','Mar','Apr','May','Jun',
   #                                                      'Jul','Aug','Sep','Oct','Nov','Dec'],
   #                                                     ['01','02','03','04','05','06',
   #                                                      '07', '08','09','10','11','12'], $str)
#df = df.replace({'Jan':'01','Feb':'02','Mar':'03','Apr':'04','May':'05','Jun':'06',
#                 'Jul':'07','Aug':'08','Sep':'09','Oct':'10','Nov':'11','Dec':'12'})
print(type(df.loc[126]))
df.loc[126]
df

In [ ]:
df['transformed_date'].iloc[99]='1983-11-14'
df['transformed_date'].iloc[271]='2008-08-01'

In [ ]:
df.loc[df['transformed_date'] == 'NaT'].head()

In [2]:
def fix_date(x):
    year = x.split('-')[0]
    year =int(year)
    if year > 2020:
        year = year - 100
        return str(year)+x[4:]
    else:
        return x

def date_sorter():
    
    # Your code here
    import re
    import pandas as pd
    global df 

    r1 = '(\d*\/\d*\/\d*)'
    r2 = '(\d*\/\d*)'
    r3 = '(\d{4})'
    r4 = '(\d*\-\d*\-\d*)'

    date_form = [r1, r2, r3, r4]

    for i in range(0, len(df)):
        for regex in date_form:
            x = re.findall(regex , df.loc[i, 'content'])
            try:
                 df.loc[i,'transformed_date']= pd.Timestamp(x[0])
            except:
                continue
    df['transformed_date'] = df['transformed_date'].astype(str)  
    df['transformed_date'].iloc[99]='1983-11-14'
    df['transformed_date'].iloc[271]='2008-08-01'
    df['transformed_date'] = df['transformed_date'].apply(lambda x : fix_date(x))
    for i in range(0,len(df)):
        df['content'][i]=df['content'][i].replace('January','01')
        df['content'][i]=df['content'][i].replace('February','02')
        df['content'][i]=df['content'][i].replace('March','03')
        df['content'][i]=df['content'][i].replace('April','04')
        df['content'][i]=df['content'][i].replace('May','05')
        df['content'][i]=df['content'][i].replace('June','06')
        df['content'][i]=df['content'][i].replace('July','07')
        df['content'][i]=df['content'][i].replace('August','08')
        df['content'][i]=df['content'][i].replace('September','09')
        df['content'][i]=df['content'][i].replace('October','10')
        df['content'][i]=df['content'][i].replace('November','11')
        df['content'][i]=df['content'][i].replace('December','12')
        df['content'][i]=df['content'][i].replace('Jan','01')
        df['content'][i]=df['content'][i].replace('Feb','02')
        df['content'][i]=df['content'][i].replace('Mar','03')
        df['content'][i]=df['content'][i].replace('Apr','04')
        df['content'][i]=df['content'][i].replace('May','05')
        df['content'][i]=df['content'][i].replace('Jun','06')
        df['content'][i]=df['content'][i].replace('Jul','07')
        df['content'][i]=df['content'][i].replace('Aug','08')
        df['content'][i]=df['content'][i].replace('Sep','09')
        df['content'][i]=df['content'][i].replace('Oct','10')
        df['content'][i]=df['content'][i].replace('Nov','11')
        df['content'][i]=df['content'][i].replace('Dec','12')
    for i in range(0, len(df)):
        x = re.findall('\d+\s\d{4}' , df.loc[i, 'content'])
        x = str(x)
        #print(x)
        if len(x) >2:
            y = x[5:9]+'-'+x[2:4]+'-'+'01'
            df.loc[i,'transformed_date'] = y
            #print(i, y, x[5:9], x[2:4])
    
    for i in range(0, len(df)):
        x = re.findall('\d+\s\d+\s\d{4}' , df.loc[i, 'content'])
        x = str(x)  
    #print(x)
        if len(x) >2:
            if int(x[5:7])<= 12:
                #print(x, type(x))
                y = x[8:12]+'-'+x[5:7]+'-'+x[2:4]
                df.loc[i,'transformed_date'] = y
                #print(i, y, x[8:12], x[5:7], x[2:4])
            else:
                y = x[8:12]+'-'+x[2:4]+'-'+x[5:7]
                df.loc[i,'transformed_date'] = y
                #print(i, y)
    #06 13, 2011 
    for i in range(0, len(df)):
        x = re.findall('\d+\s\d+\,\s\d{4}' , df.loc[i, 'content'])
        x = str(x)  
        #print(x)
        if len(x) >2:
            if int(x[5:7])<= 12:
                #print(x, type(x))
                y = x[9:13]+'-'+x[5:7]+'-'+x[2:4]
                df.loc[i,'transformed_date'] = y
                #print(i, y, x[8:12], x[5:7], x[2:4])
            else:
                y = x[9:13]+'-'+x[2:4]+'-'+x[5:7]
                df.loc[i,'transformed_date'] = y
                #print(i, y)
    #11/1984
    for i in range(0, len(df)):
        x = re.findall('(\d?\d\/\d{4})' , df.loc[i, 'content'])
        #print(i, x, len(x))

        if len(x)>0:
            x = str(x).split('/')
            #print( x[0], x[1])
            if len(x[0])<=3: a = '0'+x[0][-1]
            else: a = x[0][2:4]
            
            y = x[1][:-2]+'-'+a+'-'+'01'
            df.loc[i,'transformed_date'] = y
            #print(y)
    #5/24/1990
    for i in range(0, len(df)):
        x = re.findall('(\d?\d\/\d?\d\/\d{4})' , df.loc[i, 'content'])
        #print(i, x, len(x))
        if len(x)>=1:
            x = str(x).split('/')
            if len(x[0])<=3: a = '0'+x[0][-1]
            else: a = x[0][2:4]
            if len(x[1])<=1: b = '0'+x[1]
            else: b = x[1]
            y = x[2][0:4]+'-'+a+'-'+b
            df.loc[i,'transformed_date'] = y
            #print(y)
    

    df['transformed_date'].iloc[490]='2007-01-01'    
    df['transformed_date'] = pd.to_datetime(df['transformed_date'], format='%Y-%m-%d')
    df = df.sort_values(by='transformed_date', ascending=True)
    df = df.reset_index()
    x = df['index']
    
    return x

In [3]:
date_sorter()

0        9
1       84
2        2
3       53
4       28
      ... 
495    231
496    141
497    186
498    161
499    413
Name: index, Length: 500, dtype: int64